In [1]:
!pip install -q aixplain composio


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 600.2/600.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.2/90.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.5/210.5 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.7/44.7 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 3.2 MB/s eta 0:00:00


In [2]:
import os
from google.colab import userdata

try:
    api_key = userdata.get('AIXPLAIN_API_KEY')
    os.environ["AIXPLAIN_API_KEY"] = api_key
    print("Aixplain API Key identified successfully")
except Exception as e:
    print(f"Make sure to add the API Key and Grant access {e}")

Aixplain API Key identified successfully


In [3]:
import os
from google.colab import userdata

try:
    api_key = userdata.get('COMPOSIO_API_KEY')
    os.environ["COMPOSIO_API_KEY"] = api_key
    print("Composio API Key identified successfully")
except Exception as e:
    print(f"Make sure to add the API Key and Grant access {e}")


Composio API Key identified successfully


In [4]:
from aixplain.factories import ModelFactory
import json

### Tsting Tavily tool
NATIVE_SEARCH_ID = "6931bdf462eb386b7158def3"

print(f"Testing Tavily Search (ID: {NATIVE_SEARCH_ID})...")

try:
    native_search_tool = ModelFactory.get(NATIVE_SEARCH_ID)
    test_query = "Top manufacturing companies using AI for predictive maintenance in Europe 2025"

    structured_inputs = {"query": test_query}

    response = native_search_tool.run(inputs=structured_inputs, action="search")

    print("Tavily Search is working perfectly!")
    print("\n--- Full Live Search Results ---")

    if hasattr(response, 'to_dict'):
        full_data = response.to_dict()
    else:
        full_data = response

    print(json.dumps(full_data, indent=4, ensure_ascii=False))
    print("-" * 50)

except Exception as e:
    print(f"Test Failed: {e}")

Testing Tavily Search (ID: 6931bdf462eb386b7158def3)...
Tavily Search is working perfectly!

--- Full Live Search Results ---
{
    "status": "SUCCESS",
    "data": {
        "query": "Top manufacturing companies using AI for predictive maintenance in Europe 2025",
        "follow_up_questions": null,
        "answer": null,
        "images": [
            {
                "url": "https://miro.medium.com/max/3178/1*kgO8uyTyv-6iv4V_imah1Q.png",
                "description": "A table listing the top 10 industrial AI use cases in Europe in 2019, highlighting predictive maintenance as the leading application with a focus on manufacturing process optimization, quality assurance, and supply chain management."
            },
            {
                "url": "https://www.devopsschool.com/blog/wp-content/uploads/2025/09/1_compressed-6-768x578.jpg",
                "description": "A technician with glasses is working on a laptop and inspecting equipment in a data center with racks of serve

In [27]:
DISCOVERY_AGENT_INSTRUCTIONS = """
You are an End-User Discovery Agent.

Your responsibility is to discover, evaluate, and rank REAL companies
for any given business use-case.

You focus ONLY on company-level discovery.
You do NOT provide decision-makers, contacts, or personal data.

════════════════════════════
TOOLING CONSTRAINTS (STRICT)
════════════════════════════
- You MUST use Tavily Web Search as the ONLY discovery tool.
- You MUST perform ONLY ONE search query per request.
- You MUST NOT retry a failed search more than once.
- You MUST use GPT-4o Mini ONLY for reasoning and ranking.
- You MUST STOP after producing the final JSON output.

════════════════════════════
TAVILY USAGE RULES (CRITICAL)
════════════════════════════
When calling Tavily web_search:
Use EXACTLY this input format:

{
  "query": "<search query>"
}

Do NOT include extra parameters.
Do NOT call Tavily more than once.

════════════════════════════
WORKFLOW (MANDATORY)
════════════════════════════
1. Read the use-case carefully (Domain + Task + Target customers).
2. Form ONE precise search query that targets real companies
   providing solutions for this domain/task.
3. Call Tavily web_search ONCE.
4. Extract candidate companies mentioned in the search results.
5. Validate companies before scoring (see rules below).
6. Rank and separate results into Brand Matches vs Category Matches.
7. Output ONLY the final strict JSON.

════════════════════════════
COMPANY VALIDATION RULES (MANDATORY)
════════════════════════════
Only include an entity if ALL are true:

- It is a REAL identifiable company or startup
- The name is specific (not generic)
- There is clear web evidence it provides a relevant product/service
- It has an official website, company profile, or credible mentions

Do NOT include:

- Generic concepts or programs (e.g., "Smart Platform", "Digital Twin")
- Broad industry terms instead of companies
- Ambiguous names with no clear identification
- Research projects, unnamed initiatives, or tools without a company behind them

════════════════════════════
MATCHING LOGIC (ARD REQUIREMENT)
════════════════════════════
You MUST separate companies into two groups:

1. Brand Matches:
   - Directly solve the exact task described in the use-case
   - Strong evidence they operate in this specific domain
   - Clear product-market fit with the target customers

2. Category Matches:
   - Indirect or adjacent fit
   - Relevant industry players or enabling technologies
   - Useful alternatives but not exact domain leaders

════════════════════════════
RANKING + SCORING CRITERIA
════════════════════════════
Score each company from 0–10 based ONLY on web evidence:

- Domain relevance (most important)
- Alignment with the target customers/buyers
- Market presence / credibility
  (estimated by how frequently the company appears online)
- Innovation / sustainability fit (if applicable)

Score guidance:
- 9–10 → Perfect Brand Match, strong evidence + strong presence
- 7–8  → Strong fit, credible company
- 5–6  → Category-level match, weaker or indirect evidence
- <5   → Do NOT include

════════════════════════════
EVIDENCE REQUIREMENT (CRITICAL)
════════════════════════════
For every company, signals MUST come from the search results.
Signals must reflect:

- Products, services, or keywords mentioned online
- Proof of relevance to the domain/task
- Credible indicators of market presence

Do NOT invent signals.

════════════════════════════
OUTPUT FORMAT (STRICT JSON ONLY)
════════════════════════════
Return ONLY valid JSON:

{
  "brand_matches": [
    {
      "company": "Company Name",
      "relevance_score": 0-10,
      "signals": ["evidence_1", "evidence_2"],
      "reason": "Justification based strictly on ranking criteria"
    }
  ],
  "category_matches": [
    {
      "company": "Company Name",
      "relevance_score": 0-10,
      "signals": ["evidence_1", "evidence_2"],
      "reason": "Justification based strictly on ranking criteria"
    }
  ]
}

Rules:
- Maximum 5 companies total combined
- At least 2 Brand Matches if possible
- At least 1 Category Match if possible
- relevance_score MUST be an integer
- signals MUST be arrays of strings
- No explanations outside JSON

"""


In [28]:
from aixplain.factories import AgentFactory, ModelFactory

try:
    tavily_model = ModelFactory.get("6931bdf462eb386b7158def3")


    tavily_tool = AgentFactory.create_model_tool(
        model=tavily_model,
        name="web_search",
        description="Web search tool for discovering companies using Tavily.",
        parameters=[
            {
                "name": "query",
                "description": "The web search query string",
                "required": True,
                "type": "string"
            }
        ]
    )

    discovery_agent = AgentFactory.create(
        name="End-User Discovery Agent V3",
        description="Ranked company discovery using Tavily Search",
        instructions=DISCOVERY_AGENT_INSTRUCTIONS,
        tools=[tavily_tool],
        llm="669a63646eb56306647e1091"
    )

    print(f"Agent created successfully! ID: {discovery_agent.id}")

except Exception as e:
    print(f"Creation Failed: {e}")


Agent created successfully! ID: 6983514e7545a7b07139c797


In [29]:
USE_CASE_INPUT = """
Domain: Smart Construction Technology

Task: Identify European companies developing AI + IoT solutions
for monitoring construction sites in real-time.

Target: Large construction firms and infrastructure developers in Europe
looking to reduce safety risks and improve project efficiency.
"""


In [30]:
import json
import time


print("Running End-User Discovery Agent...")
start_time = time.time()

try:
    response = discovery_agent.run(USE_CASE_INPUT)

    latency = time.time() - start_time

    print("\n Agent finished successfully!")
    print(f"Time taken: {latency:.2f} seconds")
    print("-" * 60)

    final_result = response.data.output

    if isinstance(final_result, str):
        final_result = json.loads(final_result)

    print("Final Agent Output (Strict JSON):\n")
    print(json.dumps(final_result, indent=4, ensure_ascii=False))

except Exception as e:
    print("Agent failed with error:")
    print(e)

Running End-User Discovery Agent...
🤖  End-User Discovery Agent V3 | ⏳ Initialization
⚙️  End-User Discovery Agent V3 | GPT-4o Mini | ⏳
🤖  End-User Discovery Agent V3 | ⏳ Formatting Output
✅ Done | (15.5 s total | 4 API calls | $0.0018958499999999997)

 Agent finished successfully!
Time taken: 17.41 seconds
------------------------------------------------------------
Final Agent Output (Strict JSON):

{
    "brand_matches": [
        {
            "company": "WatchBuilt",
            "relevance_score": 9,
            "signals": [
                "Latvian startup offering IoT-enabled construction progress monitoring.",
                "Uses cameras and LiDAR for real-time monitoring."
            ],
            "reason": "WatchBuilt directly addresses the task of real-time monitoring in construction with specific IoT solutions, making it a strong brand match."
        },
        {
            "company": "MODE Inc.",
            "relevance_score": 8,
            "signals": [
            